In [1]:
import pandas as pd

## Create a mapping for fips codes, county names, states, and state ID's

In [3]:
fips_state = pd.read_csv("data/fips-by-state.csv", encoding='unicode_escape')
fips_county = pd.read_csv("data/us-state-ansi-fips.csv")
fips_county.columns = fips_county.columns.str.replace(' ', '')
fips_county['stusps'] = fips_county['stusps'].str.strip()
fips_county['stname'] = fips_county['stname'].str.strip()
merged_index = pd.merge(fips_state, fips_county, left_on="state", right_on="stusps").drop("stusps", axis=1)
merged_index

,fips,name,state,stname,st
0,1001,Autauga County,AL,Alabama,1
1,1003,Baldwin County,AL,Alabama,1
2,1005,Barbour County,AL,Alabama,1
3,1007,Bibb County,AL,Alabama,1
4,1009,Blount County,AL,Alabama,1
...,...,...,...,...,...
3138,56037,Sweetwater County,WY,Wyoming,56
3139,56039,Teton County,WY,Wyoming,56
3140,56041,Uinta County,WY,Wyoming,56
3141,56043,Washakie County,WY,Wyoming,56


### Create helper functions

In [33]:
def get_state_name_from_fips(statefips):
    return merged_index[merged_index['st']==statefips]['state'].value_counts().keys()[0]

def get_county_name_from_fips(countyfips):
    return merged_index[merged_index['fips']==countyfips]['name'].values[0]

In [34]:
# get_state_name_from_fips(6)
# get_county_name_from_fips(1001)

'Autauga County'

In [6]:
pm_CA_2011 = pd.read_csv("data/pm_conc_data_CA.csv") # fips = 6
pm_OR_2011 = pd.read_csv("data/pm_conc_data_OR.csv") # fips = 
pm_AZ_2011 = pd.read_csv("data/pm_conc_data_AZ.csv")
pm_NV_2011 = pd.read_csv("data/pm_conc_data_NV.csv")
pm_all_2011 = pd.concat([pm_CA_2011, pm_OR_2011, pm_AZ_2011, pm_NV_2011], axis=0)

Add relevant metadata columns for easy navigation.

In [35]:
pm_all_2011['state_acronym'] = pm_all_2011['statefips'].apply(get_state_name_from_fips)
pm_all_2011['county_name'] = pm_all_2011['countyfips'].apply(get_county_name_from_fips)

In [36]:
pm_all_2011

,Unnamed: 0,year,statefips,countyfips,ds_pm_pred,ds_pm_stdd,state_acronym,county_name
0,0,2011,6,6001,9.900472,1.247647,CA,Alameda County
1,1,2011,6,6003,6.121720,1.221335,CA,Alpine County
2,2,2011,6,6005,7.685671,1.298767,CA,Amador County
3,3,2011,6,6007,7.720789,1.292654,CA,Butte County
4,4,2011,6,6009,7.490275,1.297768,CA,Calaveras County
...,...,...,...,...,...,...,...,...
12,12,2011,32,32027,5.736513,1.060279,NV,Pershing County
13,13,2011,32,32029,6.890699,1.220737,NV,Storey County
14,14,2011,32,32031,7.210372,1.297015,NV,Washoe County
15,15,2011,32,32033,5.566105,1.016151,NV,White Pine County


In [38]:
pm_all_2011[pm_all_2011['state_acronym']=='CA'].sort_values(by='ds_pm_pred', ascending=False)

,Unnamed: 0,year,statefips,countyfips,ds_pm_pred,ds_pm_stdd,state_acronym,county_name
9,9,2011,6,6019,13.338245,1.677026,CA,Fresno County
15,15,2011,6,6031,13.135062,2.016612,CA,Kings County
53,53,2011,6,6107,12.934823,1.916422,CA,Tulare County
14,14,2011,6,6029,12.263347,1.546409,CA,Kern County
18,18,2011,6,6037,11.875170,0.953913,CA,Los Angeles County
35,35,2011,6,6071,11.176266,0.940509,CA,San Bernardino County
29,29,2011,6,6059,11.033006,0.920220,CA,Orange County
19,19,2011,6,6039,10.969825,1.665326,CA,Madera County
23,23,2011,6,6047,10.399814,1.607555,CA,Merced County
32,32,2011,6,6065,10.211818,0.941371,CA,Riverside County


In [53]:
demographic_data = pd.read_csv("data/demographic_data.csv")
demographic_data['County Name'] = [x[0] for x in demographic_data['County Name'].str.split(",")]

In [54]:
demographic_counties = set(demographic_data['County Name'])
pm_counties = set(pm_all_2011['county_name'])
missing_counties = len(pm_counties - demographic_counties)
missing_counties_percentage = missing_counties / len(pm_all_2011)

In [55]:
missing_counties_percentage

0.42857142857142855

Maybe there are missing counties as a function of the variables I queried for from the ACS data? I need to explore the ACS data features in more depth. For now, we can work with the counties that are represented.

In [60]:
def merge_datasets(pm, dem):
    merged = pm.merge(right=dem, left_on="county_name", right_on="County Name")
    merged = merged.drop(['state', 'county', 'County Name'], axis=1)
    return merged

In [61]:
merged = merge_datasets(pm_all_2011, demographic_data)
merged

,Unnamed: 0_x,year,statefips,countyfips,ds_pm_pred,ds_pm_stdd,state_acronym,county_name,Unnamed: 0_y,Percent_EduAttain_25y_over,Percent_BornInUS,Percent_InLaborForce,Percent_AllParentsInLaborForce
0,0,2011,6,6001,9.900472,1.247647,CA,Alameda County,0,1039030.0,67.7,65.9,64.5
1,3,2011,6,6007,7.720789,1.292654,CA,Butte County,1,140947.0,91.3,55.8,59.5
2,6,2011,6,6013,9.463319,1.227127,CA,Contra Costa County,2,716292.0,75.5,65.4,63.5
3,8,2011,6,6017,7.378507,1.279765,CA,El Dorado County,3,126849.0,89.4,62.8,62.7
4,9,2011,6,6019,13.338245,1.677026,CA,Fresno County,4,555181.0,76.6,61.4,59.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,11,2011,4,4021,7.527964,1.094924,AZ,Pinal County,7,250514.0,89.4,49.6,52.0
67,13,2011,4,4025,5.993673,1.014849,AZ,Yavapai County,8,157899.0,92.4,50.5,59.0
68,14,2011,4,4027,7.216143,0.979470,AZ,Yuma County,9,122782.0,74.7,52.0,61.8
69,1,2011,32,32003,8.345318,1.404623,NV,Clark County,0,1300342.0,76.5,65.9,63.9


In [51]:
demographic_data

,Unnamed: 0,County Name,Percent_EduAttain_25y_over,Percent_BornInUS,Percent_InLaborForce,Percent_AllParentsInLaborForce,state,county
0,0,"Alameda County, California",1039030.0,67.7,65.9,64.5,6.0,1.0
1,1,"Butte County, California",140947.0,91.3,55.8,59.5,6.0,7.0
2,2,"Contra Costa County, California",716292.0,75.5,65.4,63.5,6.0,13.0
3,3,"El Dorado County, California",126849.0,89.4,62.8,62.7,6.0,17.0
4,4,"Fresno County, California",555181.0,76.6,61.4,59.3,6.0,19.0
...,...,...,...,...,...,...,...,...
62,7,"Pinal County, Arizona",250514.0,89.4,49.6,52.0,4.0,21.0
63,8,"Yavapai County, Arizona",157899.0,92.4,50.5,59.0,4.0,25.0
64,9,"Yuma County, Arizona",122782.0,74.7,52.0,61.8,4.0,27.0
65,0,"Clark County, Nevada",1300342.0,76.5,65.9,63.9,32.0,3.0
